In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from prompt import create_prompt, UNHAPPY_PROMPT, HAPPY_CATEGORIAS_PROMPT
from main import read_csv_to_string, happy_test, unhappy_test

load_dotenv()

fatura = read_csv_to_string('gastos_cartao_credito.csv')
unhappy_prompt = create_prompt(UNHAPPY_PROMPT,fatura)
happy_prompt = create_prompt(HAPPY_CATEGORIAS_PROMPT,fatura)

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("A chave da API não foi encontrada")

client = OpenAI(api_key=api_key)


# Happy Test

In [ ]:
happy_test(happy_prompt)

# Unhappy Test

In [ ]:
unhappy_test(client, unhappy_prompt)